## Padrón Comunidad de Madrid

In [1]:
#First create a spark session and import pyspark library

import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CSV to parquet").getOrCreate()

In [2]:
#Import fuctions library
from pyspark.sql import functions as pyf

In [3]:
#Read csv file without schemas
df = spark.read.options(header=True, delimiter=';', inferSchema=True) \
    .csv('padron_madrid/Rango_Edades_Seccion_202011.csv')

df.show(5)



+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           0|               5|            null|                 1|              null|
|           1|CENTRO              |            101|PALACIO             |         1|            1001|          1|           1|               2|               3|              null|                 5|
|         

In [9]:
#Replace null values with 0
df_ = df.na.fill(0)
df_.printSchema()

root
 |-- COD_DISTRITO: integer (nullable = true)
 |-- DESC_DISTRITO: string (nullable = true)
 |-- COD_DIST_BARRIO: integer (nullable = true)
 |-- DESC_BARRIO: string (nullable = true)
 |-- COD_BARRIO: integer (nullable = true)
 |-- COD_DIST_SECCION: integer (nullable = true)
 |-- COD_SECCION: integer (nullable = true)
 |-- COD_EDAD_INT: integer (nullable = true)
 |-- EspanolesHombres: integer (nullable = true)
 |-- EspanolesMujeres: integer (nullable = true)
 |-- ExtranjerosHombres: integer (nullable = true)
 |-- ExtranjerosMujeres: integer (nullable = true)



In [14]:
#Max y Min fuctions

df_.select(pyf.max('EspanolesHombres'), pyf.min('EspanolesHombres')).show(truncate=False)

+---------------------+---------------------+
|max(EspanolesHombres)|min(EspanolesHombres)|
+---------------------+---------------------+
|102                  |0                    |
+---------------------+---------------------+



In [15]:
df_.select(pyf.max('EspanolesMujeres'), pyf.min('EspanolesMujeres')).show(truncate=False)

+---------------------+---------------------+
|max(EspanolesMujeres)|min(EspanolesMujeres)|
+---------------------+---------------------+
|93                   |0                    |
+---------------------+---------------------+



In [17]:
#Mean fuction, españoles y extranjeros

df_.select(pyf.mean('EspanolesHombres'), pyf.mean('EspanolesMujeres'), \
           pyf.mean('ExtranjerosHombres'), pyf.mean('ExtranjerosMujeres')) \
    .show()

+---------------------+---------------------+-----------------------+-----------------------+
|avg(EspanolesHombres)|avg(EspanolesMujeres)|avg(ExtranjerosHombres)|avg(ExtranjerosMujeres)|
+---------------------+---------------------+-----------------------+-----------------------+
|    5.519729116598553|    6.300522321785574|      1.026587483637986|     1.1853547873885173|
+---------------------+---------------------+-----------------------+-----------------------+



In [19]:
#Sum fuction

df_.select(pyf.sum('EspanolesHombres'), pyf.sum('EspanolesMujeres'), \
           pyf.sum('ExtranjerosHombres'), pyf.sum('ExtranjerosMujeres')) \
    .show()

+---------------------+---------------------+-----------------------+-----------------------+
|sum(EspanolesHombres)|sum(EspanolesMujeres)|sum(ExtranjerosHombres)|sum(ExtranjerosMujeres)|
+---------------------+---------------------+-----------------------+-----------------------+
|              1311449|              1496960|                 243910|                 281632|
+---------------------+---------------------+-----------------------+-----------------------+



In [22]:
#Total population: espanoles y extranjeros

df_.select((pyf.sum('EspanolesHombres') + pyf.sum('EspanolesMujeres')), \
           (pyf.sum('ExtranjerosHombres') + pyf.sum('ExtranjerosMujeres'))).show()

+-----------------------------------------------+---------------------------------------------------+
|(sum(EspanolesHombres) + sum(EspanolesMujeres))|(sum(ExtranjerosHombres) + sum(ExtranjerosMujeres))|
+-----------------------------------------------+---------------------------------------------------+
|                                        2808409|                                             525542|
+-----------------------------------------------+---------------------------------------------------+



In [28]:
print("Total population in Madrid: " + \
    str(df_.select(pyf.sum('EspanolesHombres') + pyf.sum('EspanolesMujeres') \
           + pyf.sum('ExtranjerosHombres') + pyf.sum('ExtranjerosMujeres')).collect()[0][0]))

Total population in Madrid: 3333951


.- Next steps for the analysis: outliers
.- At this point it seems that that max and min are out of range. Is it possible 93 women living at the same place?